# Data Science Project: Planning Stage (Individual)

**Name:** Victoria Chen  
**Student Number:** 66263492  
**Section:** 004  
**Group:** 30  

## 1. Data Description

In [1]:
# Load tidyverse package
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
# Load dataset
players <- read_csv("players.csv")
glimpse(players)

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 196
Columns: 7
$ experience   <chr> "Pro", "Veteran", "Veteran", "Amateur", "Regular", "Amate…
$ subscribe    <lgl> TRUE, TRUE, FALSE, TRUE, TRUE, TRUE, TRUE, FALSE, TRUE, T…
$ hashedEmail  <chr> "f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8…
$ played_hours <dbl> 30.3, 3.8, 0.0, 0.7, 0.1, 0.0, 0.0, 0.0, 0.1, 0.0, 1.6, 0…
$ name         <chr> "Morgan", "Christian", "Blake", "Flora", "Kylie", "Adrian…
$ gender       <chr> "Male", "Male", "Male", "Female", "Male", "Female", "Fema…
$ Age          <dbl> 9, 17, 17, 21, 21, 17, 19, 21, 47, 22, 23, 17, 25, 22, 17…


### Basic Information

The `players.csv` data was collected by The Pacific Laboratory for Artificial Intelligence (PLAI), a research group based in the Department of Computer Science at the University of British Columbia. By setting up the Minecraft server, PLAICraft, they were able to collect data about players and how they interacted with the game. This dataset contains player-level information, with each observation representing one unique player.

From reading in the dataset, we observe the following: 
- Number of observations (rows): 196  
- Number of variables (columns): 7

| Variable | Type | Description | Possible values (if categorical) | Notes |
| -- | -- | -- | -- | -- |
| experience | chr | Player's experience level | Beginner, Amateur, Regular, Pro, Veteran | Experience level may be inconsistent due to subjectivity |
| subscribe | lgl | Whether the player is subscribed to the newsletter | TRUE, FALSE | Will serve as the target variable later
| hashedEmail | chr | Player's email (hashed) | | Can be used as a unique identifier |
| played_hours | dbl | Number of hours the player has played on the server | | May contain outliers (e.g. zeros for inactive players)
| name | chr | Player's name | | May not be unique
| gender | chr | Player's gender | Male, Female, Non-binary, Two-Spirited, Agender, Other, Prefer not to say | Lack of observations for some categories
| Age | dbl | Player's age in years | | Can be grouped in increments

### Summary Statistics

In [13]:
# Factorize categorical variables
players <- players |>
  mutate(
    experience = factor(experience,
                        levels = c("Beginner", "Amateur", "Regular", "Pro", "Veteran"),
                        ordered = TRUE),
    gender = factor(gender)
  )

# Calculate summary statistics for each variable
summary(players)

    experience subscribe       hashedEmail         played_hours    
 Beginner:35   Mode :logical   Length:196         Min.   :  0.000  
 Amateur :63   FALSE:52        Class :character   1st Qu.:  0.000  
 Regular :36   TRUE :144       Mode  :character   Median :  0.100  
 Pro     :14                                      Mean   :  5.846  
 Veteran :48                                      3rd Qu.:  0.600  
                                                  Max.   :223.100  
                                                                   
      name                   gender         Age       
 Aarav  :  1   Agender          :  2   Min.   : 9.00  
 Aaron  :  1   Female           : 37   1st Qu.:17.00  
 Adrian :  1   Male             :124   Median :19.00  
 Ahmed  :  1   Non-binary       : 15   Mean   :21.14  
 Aiden  :  1   Other            :  1   3rd Qu.:22.75  
 Akio   :  1   Prefer not to say: 11   Max.   :58.00  
 (Other):190   Two-Spirited     :  6   NA's   :2      

Using the `summary` command, we notice that:
- The most prevalent experience level of players is "Amateur", followed by "Veteran"
- The proportion of individuals subscribed to the newsletter is 144 / 196 = 73.47%
- The average number of hours played is 5.846, yet the median is only 0.1
- The mean (21.14) and median (19.00) ages are both approximately 20 years old

## 2. Questions

## 3. Exploratory Data Analysis and Visualization

## 4. Methods and Plan